In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load data

In [ ]:
df_offer = pd.read_csv("data/portfolio.csv")
df_offer.head()

In [ ]:
df_customer = pd.read_csv("data/profile.csv")
df_customer.head()

In [ ]:
df_transcript = pd.read_csv("data/transcript.csv")
df_transcript.head()

## Profile data

In [ ]:
df_offer.columns

In [ ]:
df_customer.columns

In [ ]:
df_transcript.columns

## Clean data

### Drop unnamed column

In [ ]:
df_offer = df_offer.drop('Unnamed: 0', axis = 1)
df_customer = df_customer.drop('Unnamed: 0', axis = 1)
df_transcript = df_transcript.drop('Unnamed: 0', axis = 1)

### Check missing value

In [ ]:
df_offer.isna().sum()

In [ ]:
df_customer.isna().sum()

In [ ]:
df_transcript.isna().sum()

In [ ]:
missing_gender = df_customer[df_customer['gender'].isna()]
missing_income = df_customer[df_customer['income'].isna()]

np.sum(missing_gender['id'] == missing_income['id'])

### Remove rows with missing values in df_customer

In [ ]:
id_to_remove = missing_income['id']
df_customer_no_na = df_customer[~df_customer['id'].isin(id_to_remove)]
df_customer_no_na = df_customer_no_na.reset_index(drop = True)
df_customer_no_na.isna().sum()

### Check data types

In [ ]:
df_customer_no_na['became_member_on'] = pd.to_datetime(df_customer_no_na['became_member_on'].astype(str), format = '%Y%m%d')
df_customer_no_na['became_member_on'].head()

In [ ]:
df_transcript.rename(columns = {'time' : 'hours_since_start'}, inplace = True)
df_transcript.head(1)

## Data Transformation

In [ ]:
# Elements in value_columns are strings. Convert them to dictionaries
value_column = df_transcript['value']
value_column = value_column.apply(eval)
value_column[:3]

In [ ]:
# Extract the keys
dictionary_key_column = [list(d.keys())[0] for d in value_column]
# Extract the values
dictionary_value_column = [list(d.values())[0] for d in value_column]
# Create a df containing the keys and values of the dictionary elements of `value_column`
value_column_split = pd.DataFrame(columns = ['dict_key', 'dict_value'])
value_column_split['dict_key'] = dictionary_key_column
value_column_split['dict_value'] = dictionary_value_column
value_column_split.head(3)

In [ ]:
# Replace `value` column with df `value_column_split`
df_transcript_value_mod = df_transcript.drop('value', axis = 1)
df_transcript_value_mod = pd.concat([df_transcript_value_mod, value_column_split], axis = 1)

In [ ]:
df_transcript_value_mod.columns

### Save checkpoints after data cleaning

In [ ]:
offers = df_offer.copy()
customers = df_customer_no_na.copy()
transcripts = df_transcript_value_mod.copy()

## Exploratory Data Analysis (EDA)

### Univariate Analysis - gender

In [ ]:
count = customers['gender'].value_counts()
label = ['Male', 'Female', 'Other']
plt.figure(figsize=(4,3))
plt.pie(x = count, labels = label, autopct = '%1.1f%%')
plt.title('Gender', fontsize = 12)
plt.show()

### Univariate Analysis - age

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (12,4))
sns.boxplot(data = customers, x = 'age', ax = axs[0])
sns.histplot(data = customers, x = 'age', ax = axs[1])
axs[0].xaxis.label.set_size(15)
axs[1].xaxis.label.set_size(15)
axs[1].yaxis.label.set_size(15)

print(customers['age'].describe())

In [ ]:
# Create a new column `age_group`
age_group = pd.cut(customers["age"], bins = [customers["age"].min(), 26, 36, 46, 56, 66, 76, 86, customers["age"].max()], labels=['18-25ys', '26-35ys', '36-45ys', '46-55ys', '56-65ys', '66-75ys', '76-85ys', '> 86ys'])
customers["age_group"] = age_group

# Visualize customers by age groups
age_group_percentage = (customers['age_group'].value_counts().sort_index() / customers.shape[0] * 100).round(1)

plt.xlabel("Age")
plt.ylabel("Percentage %")
plt.title("Age Distribution")
bar_plot = plt.bar(x = age_group_percentage.index.values, height = age_group_percentage)
for i,bar in enumerate(bar_plot.patches):
    x, y = bar.get_xy()
    plt.text(x+bar.get_width()/2, y+bar.get_height()+0.2,
            str(age_group_percentage.iloc[i]) + '%',
            ha = 'center', weight = 'bold')

plt.tight_layout()
plt.show()

### Univariate Analysis - income

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (12,4))
sns.boxplot(data = customers, x = 'income', ax = axs[0])
sns.histplot(data = customers, x = 'income', ax = axs[1])
axs[0].xaxis.label.set_size(15)
axs[1].xaxis.label.set_size(15)
axs[1].yaxis.label.set_size(15)

print(customers['income'].describe())

In [ ]:
# Create a new column `income_group`
income_group = pd.cut(customers['income'],
      bins = [customers['income'].min(), 45000, 60000, 75000, 90000, 105000, customers['income'].max()],
      labels = ['30-45k', '45-60k', '60-75k', '75-90k', '90-105k', '> 105k'])
customers['income_group'] = income_group

# Visualize customers by income groups
income_group_percentage = (customers['income_group'].value_counts().sort_index() / customers.shape[0] * 100).round(1)

plt.xlabel('Income USD')
plt.ylabel('Percentage %')
plt.title('Income Distribution')


bar_plot = plt.bar(x = income_group_percentage.index.values, height = income_group_percentage)
for i,bar in enumerate(bar_plot.patches):
    x, y = bar.get_xy()
    plt.text(x+bar.get_width()/2, y+bar.get_height()+0.2,
            str(income_group_percentage.iloc[i]) + '%',
            ha = 'center', weight = 'bold')

plt.tight_layout()
plt.show()

### Univariate Analysis - became_member_on

In [ ]:
df_member = customers[['id', 'became_member_on']]
df_member = df_member.assign(year = df_member.became_member_on.dt.year,
                             month = df_member.became_member_on.dt.month,
                             day = df_member.became_member_on.dt.day,
                             month_name = df_member.became_member_on.dt.month_name(),
                             day_of_week = df_member.became_member_on.dt.weekday,
                             day_name = df_member.became_member_on.dt.day_name())
df_member.head(3)

In [ ]:
sns.countplot(data=df_member, x="year")
plt.title('Number of New Members by Year')
plt.xlabel('Year')
plt.ylabel('Count')
plt.show()

In [ ]:
sns.countplot(data=df_member.sort_values("month"), y="month_name")
plt.title('Number of New Members by Month')
plt.xlabel('Year')
plt.ylabel('Month')
plt.show()

In [ ]:
sns.countplot(data=df_member.sort_values("day_of_week"), y="day_name")
plt.title('Number of New Members by Weekday')
plt.xlabel('Year')
plt.ylabel('Day')
plt.show()

### Bivariate Analysis - age vs gender

In [ ]:
print(customers.groupby("gender").agg({"age": "mean"}))
sns.kdeplot(data=customers, x="age", hue="gender", fill=True)
plt.title("Age vs Gender")
plt.show()

### Bivariate Analysis - income vs gender

In [ ]:
print(customers.groupby("gender").agg({"income": "mean"}))
sns.kdeplot(data=customers, x="income", hue="gender", fill=True)
plt.title("Income vs Gender")
plt.show()

### Bivariate Analysis - income vs age

In [ ]:
sns.barplot(data=customers, x="age_group", y="income")
plt.title("Income vs Age")
plt.show()